In [1]:
! pip install LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.0 MB/s eta 0:00:00


In [2]:
import time
import numpy as np
import pandas as pd
from libreco.data import split_by_ratio_chrono, DatasetFeat, split_by_ratio, random_split, split_by_num
from libreco.algorithms import FM, WideDeep, DeepFM, AutoInt, DIN



Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_WARNINGS"] = "FALSE"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [4]:
def reset_state(name):
    tf.compat.v1.reset_default_graph()
    print("\n", "=" * 30, name, "=" * 30)

In [13]:
import pandas as pd
#start_time = time.perf_counter()
data = pd.read_csv("patientrs.csv",sep=",", header=0)
train_data, eval_data = split_by_ratio(data, test_size=0.2)


In [14]:
data.head()

,user,item,label,person_id,disease,age,gender,DEGREE_CENTRALITY,EIGENVECTOR,BETWEENNESS_CENTRALITY,CLUSTCOEF,PAGERANK,Comorbidities,Behavioural,Mental
0,1,6,1,2,"Hypertension, uncomplicated and Hypertension, ...",108,F,1.0,0.092622,0.0,0.035508,0.024782,1,0,0
1,1,24,1,2,Fluid and electrolyte disorders,108,F,1.0,0.307246,0.0,0.074907,0.069379,1,0,0
2,2,2,1,49,Cardiac arrhythmias,117,F,1.0,0.339093,0.0,0.078110,0.074771,1,0,0
3,2,24,1,49,Fluid and electrolyte disorders,117,F,1.0,0.307246,0.0,0.074907,0.069379,1,0,0
4,2,32,1,49,Cerebrovascular disease,117,F,1.0,0.123452,0.0,0.040629,0.031130,1,0,0


In [15]:
data[data["user"]==2916]

,user,item,label,person_id,disease,age,gender,DEGREE_CENTRALITY,EIGENVECTOR,BETWEENNESS_CENTRALITY,CLUSTCOEF,PAGERANK,Comorbidities,Behavioural,Mental
8556,2916,10,1,10483,"Diabetes, uncomplicated",61,M,1.0,0.337493,0.0,0.069097,0.066129,1,0,0
8557,2916,11,1,10483,"Diabetes, complicated",61,M,1.0,0.208104,0.0,0.049615,0.042058,1,0,0
8558,2916,24,1,10483,Fluid and electrolyte disorders,61,M,1.0,0.307246,0.0,0.074907,0.069379,1,0,0
8559,2916,34,1,10483,Smoking,61,M,1.0,0.511474,0.0,0.105483,0.121583,0,1,0


In [16]:
#train_data.to_csv("patientrs_train.csv")
#eval_data.to_csv("patientrs_test.csv")

In [ ]:

    # specify complete columns information
sparse_col = [ 'age',"gender", "DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#sparse_col = ["gender",'Comorbidities','Behavioural','Mental']
dense_col = ["age"]
user_col = ["age", "gender"]
item_col = ["DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#item_col = ['Comorbidities','Behavioural','Mental']
train_data, data_info = DatasetFeat.build_trainset(train_data, user_col, item_col, sparse_col, dense_col)
eval_data = DatasetFeat.build_testset(eval_data)
print(data_info)

In [ ]:
reset_state("FM")
fm = FM("rating", data_info, embed_size=16, n_epochs=10,
            lr=1e-4, lr_decay=False, reg=None, batch_size=256,
            num_neg=1, use_bn=True, dropout_rate=None, tf_sess_config=None)
fm.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
           metrics=["rmse", "mae", "r2"])
#print("prediction: ", fm.predict(user=1, item=18))
print("recommendation: ", fm.recommend_user(user=2916, n_rec=10))


 ============================== FM ==============================
Training start time: 2021-09-22 04:45:06
total params: 337,966 | embedding params: 337,911 | network params: 55


train: 100%|██████████| 169/169 [00:00<00:00, 222.32it/s]


Epoch 1 elapsed: 0.784s
	 train_loss: 2.0519


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 25.93it/s]


	 eval rmse: 1.2804
	 eval mae: 0.8980
	 eval r2: -0.9075


...

	 eval rmse: 0.8867
	 eval mae: 0.7137
	 eval r2: 0.0852


train: 100%|██████████| 169/169 [00:00<00:00, 310.96it/s]


Epoch 10 elapsed: 0.576s
	 train_loss: 0.369


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 150.38it/s]


	 eval rmse: 0.8847
	 eval mae: 0.7028
	 eval r2: 0.0894
recommendation:  [(30, 1.5336154), (18, 1.4285427), (17, 1.3611647), (34, 1.2481079), (19, 1.2131763), (33, 0.92220896), (2, 0.90934175), (29, 0.9013186), (13, 0.8541895), (8, 0.8512492)]


In [ ]:
#data["label"] = 1  # convert to implicit data and do negative sampling afterwards

# split into train and test data based on time
train_data, test_data = split_by_ratio(data, test_size=0.2)

# specify complete columns information
sparse_col = [ 'age',"gender", "DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#sparse_col = ["gender",'Comorbidities','Behavioural','Mental']
dense_col = ["age"]
user_col = ["age", "gender"]
item_col = ["DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#item_col = ['Comorbidities','Behavioural','Mental']
train_data, data_info = DatasetFeat.build_trainset(
    train_data, user_col, item_col, sparse_col, dense_col
)
test_data = DatasetFeat.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
test_data.build_negative_samples(data_info)
print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %
reset_state("FM")
fm = FM("ranking", data_info, embed_size=16, n_epochs=20,
            lr=1e-4, lr_decay=False, reg=None, batch_size=256,
            num_neg=0, use_bn=True, dropout_rate=0, tf_sess_config=None)
fm.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"])

# predict preference of user 2211 to item 110
#print("prediction: ", fm.predict(user=2916, item=1))
# recommend 7 items for user 2916
print("recommendation(id, probability): ", fm.recommend_user(user=2916, n_rec=10))



random neg item sampling elapsed: 0.046s
random neg item sampling elapsed: 0.006s
n_users: 19828, n_items: 34, data sparsity: 6.3931 %

 ============================== FM ==============================
Training start time: 2021-09-22 05:06:13
total params: 341,984 | embedding params: 341,923 | network params: 61


train: 100%|██████████| 337/337 [00:01<00:00, 239.13it/s]


Epoch 1 elapsed: 1.448s
	 train_loss: 0.5948


eval_rec: 100%|██████████| 2048/2048 [00:01<00:00, 1111.39it/s]


	 eval log_loss: 0.5565
	 eval roc_auc: 0.7800
	 eval precision@10: 0.0813
	 eval recall@10: 0.7503
	 eval map@10: 0.6240
	 eval ndcg@10: 0.6633


...

	 eval log_loss: 0.6035
	 eval roc_auc: 0.7790
	 eval precision@10: 0.0840
	 eval recall@10: 0.7667
	 eval map@10: 0.4837
	 eval ndcg@10: 0.5595


train: 100%|██████████| 337/337 [00:01<00:00, 288.78it/s]


Epoch 20 elapsed: 1.214s
	 train_loss: 0.3924


eval_rec: 100%|██████████| 2048/2048 [00:01<00:00, 1151.93it/s]


	 eval log_loss: 0.6231
	 eval roc_auc: 0.7696
	 eval precision@10: 0.0831
	 eval recall@10: 0.7584
	 eval map@10: 0.4416
	 eval ndcg@10: 0.5248
recommendation(id, probability):  [(34, 0.94981056), (2, 0.87971807), (9, 0.75267345), (18, 0.7046006), (23, 0.67034405), (19, 0.5019464), (32, 0.38405478), (28, 0.34472543), (8, 0.30627713), (26, 0.2827779)]


In [ ]:
    reset_state("Wide_Deep")
    wd = WideDeep("rating", data_info, embed_size=16, n_epochs=20,
                  lr={"wide": 0.01, "deep": 1e-4}, lr_decay=False, reg=None,
                  batch_size=256, num_neg=1, use_bn=False, dropout_rate=None,
                  hidden_units="128,64,32", tf_sess_config=None)
    wd.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
           metrics=["rmse", "mae", "r2"])
    #print("prediction: ", wd.predict(user=1, item=2333))
    print("recommendation: ", wd.recommend_user(user=2916, n_rec=10))


 ============================== Wide_Deep ==============================
Training start time: 2021-09-22 04:59:13
total params: 362,750 | embedding params: 338,135 | network params: 24,615


train: 100%|██████████| 169/169 [00:00<00:00, 169.02it/s]


Epoch 1 elapsed: 1.024s
	 train_loss: 0.9739


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 23.65it/s]


	 eval rmse: 0.9503
	 eval mae: 0.7815
	 eval r2: -0.0507


...

	 eval rmse: 0.8723
	 eval mae: 0.7028
	 eval r2: 0.1146


train: 100%|██████████| 169/169 [00:00<00:00, 231.80it/s]


Epoch 20 elapsed: 0.761s
	 train_loss: 0.2953


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 68.38it/s]


	 eval rmse: 0.8728
	 eval mae: 0.7022
	 eval r2: 0.1136
recommendation:  [(18, 1.4362942), (30, 1.3606696), (17, 1.3321054), (19, 1.2585492), (34, 1.1712719), (33, 1.0719392), (29, 0.89466435), (20, 0.89213336), (2, 0.87747395), (13, 0.86994696)]


In [ ]:
train_data, test_data = split_by_ratio(data, test_size=0.2)

# specify complete columns information
#sparse_col = [ 'age',"gender", "DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
sparse_col = ["gender",'Comorbidities','Behavioural','Mental']
dense_col = ["age"]
user_col = ["age", "gender"]
#item_col = ["DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
item_col = ['Comorbidities','Behavioural','Mental']



train_data, data_info = DatasetFeat.build_trainset(
    train_data, user_col, item_col, sparse_col, dense_col
)
test_data = DatasetFeat.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
test_data.build_negative_samples(data_info)
print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %
reset_state("Wide_Deep")
wd = WideDeep("ranking", data_info, embed_size=16, n_epochs=20,
                  lr={"wide": 0.01, "deep": 1e-4}, lr_decay=False, reg=None,
                  batch_size=256, num_neg=1, use_bn=False, dropout_rate=None,
                  hidden_units="128,64,32", tf_sess_config=None)
wd.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"])

# predict preference of user 2211 to item 110
#print("prediction: ", fm.predict(user=2916, item=1))
# recommend 7 items for user 2916
print("recommendation(id, probability): ", wd.recommend_user(user=2916, n_rec=10))


random neg item sampling elapsed: 0.046s
random neg item sampling elapsed: 0.004s
n_users: 19828, n_items: 34, data sparsity: 6.3931 %

 ============================== Wide_Deep ==============================
Training start time: 2021-09-22 05:11:18
total params: 362,750 | embedding params: 338,135 | network params: 24,615


train: 100%|██████████| 337/337 [00:01<00:00, 204.63it/s]


Epoch 1 elapsed: 1.683s
	 train_loss: 0.6235


eval_rec: 100%|██████████| 2048/2048 [00:02<00:00, 957.20it/s]


	 eval log_loss: 0.5278
	 eval roc_auc: 0.8055
	 eval precision@10: 0.0798
	 eval recall@10: 0.7345
	 eval map@10: 0.6201
	 eval ndcg@10: 0.6565


...

	 eval log_loss: 0.5687
	 eval roc_auc: 0.7903
	 eval precision@10: 0.0853
	 eval recall@10: 0.7787
	 eval map@10: 0.6064
	 eval ndcg@10: 0.6555


train: 100%|██████████| 337/337 [00:01<00:00, 225.35it/s]


Epoch 20 elapsed: 1.540s
	 train_loss: 0.4425


eval_rec: 100%|██████████| 2048/2048 [00:02<00:00, 948.36it/s]


	 eval log_loss: 0.5936
	 eval roc_auc: 0.7708
	 eval precision@10: 0.0839
	 eval recall@10: 0.7651
	 eval map@10: 0.5652
	 eval ndcg@10: 0.6204
recommendation(id, probability):  [(34, 0.8905234), (19, 0.83848953), (2, 0.83236617), (18, 0.7930578), (9, 0.7339003), (22, 0.6386224), (30, 0.6239816), (23, 0.5420456), (26, 0.43297508), (31, 0.39628348)]


In [ ]:
reset_state("DeepFM")
deepfm = DeepFM("rating", data_info, embed_size=16, n_epochs=30,
                    lr=1e-4, lr_decay=False, reg=None, batch_size=2048,
                    num_neg=1, use_bn=False, dropout_rate=None,
                    hidden_units="128,64,32", tf_sess_config=None)
deepfm.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
               metrics=["rmse", "mae", "r2"])
#print("prediction: ", deepfm.predict(user=1, item=2333))
print("recommendation: ", deepfm.recommend_user(user=2916, n_rec=7))


 ============================== DeepFM ==============================
Training start time: 2021-09-21 08:19:07
total params: 362,767 | embedding params: 338,135 | network params: 24,632


train: 100%|██████████| 22/22 [00:00<00:00, 34.96it/s]


Epoch 1 elapsed: 0.652s
	 train_loss: 3.5593


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 20.66it/s]


	 eval rmse: 1.2919
	 eval mae: 0.8998
	 eval r2: -0.9421


...

	 eval rmse: 0.8795
	 eval mae: 0.7115
	 eval r2: 0.0999


train: 100%|██████████| 22/22 [00:00<00:00, 59.52it/s]


Epoch 30 elapsed: 0.402s
	 train_loss: 0.3328


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 41.16it/s]


	 eval rmse: 0.8815
	 eval mae: 0.7124
	 eval r2: 0.0958
recommendation:  [(30, 1.5681673), (18, 1.5471661), (17, 1.3904479), (19, 1.2498953), (34, 1.1190872), (29, 0.9994422), (20, 0.9435787)]


In [ ]:
train_data, test_data = split_by_ratio(data, test_size=0.2)

# specify complete columns information
sparse_col = [ 'age',"gender", "DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#sparse_col = ["gender",'Comorbidities','Behavioural','Mental']
dense_col = ["age"]
user_col = ["age", "gender"]
item_col = ["DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#item_col = ['Comorbidities','Behavioural','Mental']



train_data, data_info = DatasetFeat.build_trainset(
    train_data, user_col, item_col, sparse_col, dense_col
)
test_data = DatasetFeat.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
test_data.build_negative_samples(data_info)
print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %
reset_state("DeepFM")
deepfm = DeepFM("ranking", data_info, embed_size=16, n_epochs=30,
                    lr=1e-4, lr_decay=False, reg=None, batch_size=2048,
                    num_neg=1, use_bn=False, dropout_rate=None,
                    hidden_units="128,64,32", tf_sess_config=None)
deepfm.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"])

# predict preference of user 2211 to item 110
#print("prediction: ", fm.predict(user=2916, item=1))
# recommend 7 items for user 2916
print("recommendation(id, probability): ", deepfm.recommend_user(user=2916, n_rec=10))

random neg item sampling elapsed: 0.053s
random neg item sampling elapsed: 0.004s
n_users: 19828, n_items: 34, data sparsity: 6.3931 %

 ============================== DeepFM ==============================
Training start time: 2021-09-22 06:07:39
total params: 379,073 | embedding params: 342,147 | network params: 36,926


train: 100%|██████████| 43/43 [00:01<00:00, 41.57it/s]


Epoch 1 elapsed: 1.081s
	 train_loss: 0.6858


eval_rec: 100%|██████████| 2048/2048 [00:02<00:00, 890.70it/s]


	 eval log_loss: 0.6692
	 eval roc_auc: 0.7936
	 eval precision@10: 0.0873
	 eval recall@10: 0.7965
	 eval map@10: 0.6081
	 eval ndcg@10: 0.6607


...

	 eval log_loss: 0.5609
	 eval roc_auc: 0.8131
	 eval precision@10: 0.0833
	 eval recall@10: 0.7595
	 eval map@10: 0.6190
	 eval ndcg@10: 0.6612


train: 100%|██████████| 43/43 [00:00<00:00, 54.35it/s]


Epoch 30 elapsed: 0.835s
	 train_loss: 0.4899


eval_rec: 100%|██████████| 2048/2048 [00:02<00:00, 920.04it/s]


	 eval log_loss: 0.5549
	 eval roc_auc: 0.8129
	 eval precision@10: 0.0842
	 eval recall@10: 0.7671
	 eval map@10: 0.6168
	 eval ndcg@10: 0.6612
recommendation(id, probability):  [(34, 0.940829), (19, 0.85177267), (2, 0.762323), (18, 0.5894073), (9, 0.5131422), (30, 0.4105088), (22, 0.39129686), (23, 0.32068872), (8, 0.29409093), (26, 0.29104593)]


In [ ]:
    reset_state("AutoInt")
    autoint = AutoInt("rating", data_info, embed_size=16, n_epochs=30,
                      att_embed_size=(8, 8, 8), num_heads=4, use_residual=False,
                      lr=1e-3, lr_decay=False, reg=None, batch_size=2048,
                      num_neg=1, use_bn=False, dropout_rate=None,
                      hidden_units="128,64,32", tf_sess_config=None)
    autoint.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
                metrics=["rmse", "mae", "r2"])
    #print("prediction: ", autoint.predict(user=1, item=2333))
    print("recommendation: ", deepfm.recommend_user(user=2916, n_rec=7))


 ============================== AutoInt ==============================
Training start time: 2021-09-21 08:22:55


train: 100%|██████████| 22/22 [00:03<00:00,  6.63it/s]


Epoch 1 elapsed: 3.342s
	 train_loss: 3.6886


eval_pred: 100%|██████████| 1/1 [00:00<00:00,  6.29it/s]


	 eval rmse: 1.2919
	 eval mae: 0.8998
	 eval r2: -0.9421


...

	 eval rmse: 0.9008
	 eval mae: 0.6986
	 eval r2: 0.0558


train: 100%|██████████| 22/22 [00:02<00:00,  7.37it/s]


Epoch 30 elapsed: 3.012s
	 train_loss: 0.2683


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 11.06it/s]


	 eval rmse: 0.9017
	 eval mae: 0.6956
	 eval r2: 0.0540
recommendation:  [(34, 0.9379472), (19, 0.8462674), (2, 0.7604815), (18, 0.5789426), (9, 0.49785006), (30, 0.4083643), (22, 0.3768336)]


In [ ]:
train_data, test_data = split_by_ratio(data, test_size=0.2)

# specify complete columns information
sparse_col = [ 'age',"gender", "DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#sparse_col = ["gender",'Comorbidities','Behavioural','Mental']
dense_col = ["age"]
user_col = ["age", "gender"]
item_col = ["DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
#item_col = ['Comorbidities','Behavioural','Mental']



train_data, data_info = DatasetFeat.build_trainset(
    train_data, user_col, item_col, sparse_col, dense_col
)
test_data = DatasetFeat.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
test_data.build_negative_samples(data_info)
print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %
reset_state("AutoInt")
autoint = AutoInt("ranking", data_info, embed_size=16, n_epochs=30,
                      att_embed_size=(8, 8, 8), num_heads=4, use_residual=False,
                      lr=1e-3, lr_decay=False, reg=None, batch_size=2048,
                      num_neg=1, use_bn=False, dropout_rate=None,
                      hidden_units="128,64,32", tf_sess_config=None)
autoint.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"])

# predict preference of user 2211 to item 110
#print("prediction: ", fm.predict(user=2916, item=1))
# recommend 7 items for user 2916
print("recommendation(id, probability): ", autoint.recommend_user(user=2916, n_rec=7))

random neg item sampling elapsed: 0.049s
random neg item sampling elapsed: 0.004s
n_users: 19828, n_items: 34, data sparsity: 6.3931 %

 ============================== AutoInt ==============================
Training start time: 2021-09-22 06:09:55


train: 100%|██████████| 43/43 [00:08<00:00,  5.06it/s]


Epoch 1 elapsed: 8.541s
	 train_loss: 0.6931


eval_rec: 100%|██████████| 2048/2048 [00:05<00:00, 400.58it/s]


	 eval log_loss: 0.6920
	 eval roc_auc: 0.7617
	 eval precision@10: 0.0821
	 eval recall@10: 0.7514
	 eval map@10: 0.6197
	 eval ndcg@10: 0.6601


...

	 eval log_loss: 1.2137
	 eval roc_auc: 0.6818
	 eval precision@10: 0.0729
	 eval recall@10: 0.6611
	 eval map@10: 0.4309
	 eval ndcg@10: 0.4925


train: 100%|██████████| 43/43 [00:08<00:00,  4.86it/s]


Epoch 30 elapsed: 8.896s
	 train_loss: 0.2991


eval_rec: 100%|██████████| 2048/2048 [00:05<00:00, 401.53it/s]


	 eval log_loss: 1.3143
	 eval roc_auc: 0.6435
	 eval precision@10: 0.0664
	 eval recall@10: 0.6026
	 eval map@10: 0.3968
	 eval ndcg@10: 0.4526
recommendation(id, probability):  [(2, 0.74221516), (9, 0.4663789), (19, 0.36935163), (14, 0.35985485), (23, 0.3162064), (22, 0.29937503), (26, 0.2872466)]


In [ ]:
    reset_state("DIN")
    din = DIN("rating", data_info, embed_size=16, n_epochs=20,
              recent_num=10, lr=1e-4, lr_decay=False, reg=None,
              batch_size=2048, num_neg=1, use_bn=False, dropout_rate=None,
              hidden_units="128,64,32", tf_sess_config=None, use_tf_attention=True)
    din.fit(train_data, verbose=2, shuffle=True, eval_data=eval_data,
            metrics=["rmse", "mae", "r2"])
    print("prediction: ", din.predict(user=1, item=18))
    print("recommendation: ", deepfm.recommend_user(user=2916, n_rec=7))


 ============================== DIN ==============================
Training start time: 2021-09-21 08:31:50
total params: 351,057 | embedding params: 318,257 | network params: 32,800


train: 100%|██████████| 22/22 [00:01<00:00, 21.24it/s]


Epoch 1 elapsed: 1.058s
	 train_loss: 1.8254


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 16.76it/s]


	 eval rmse: 1.0688
	 eval mae: 0.8526
	 eval r2: -0.3292


...

	 eval rmse: 0.8642
	 eval mae: 0.7042
	 eval r2: 0.1310


train: 100%|██████████| 22/22 [00:00<00:00, 25.81it/s]


Epoch 20 elapsed: 0.880s
	 train_loss: 0.391


eval_pred: 100%|██████████| 1/1 [00:00<00:00, 41.28it/s]


	 eval rmse: 0.8715
	 eval mae: 0.7053
	 eval r2: 0.1162
prediction:  [1.8412524]
recommendation:  [(34, 0.9379472), (19, 0.8462674), (2, 0.7604815), (18, 0.5789426), (9, 0.49785006), (30, 0.4083643), (22, 0.3768336)]


In [ ]:
#data["label"] = 1
train_data, test_data = split_by_ratio(data, test_size=0.2)

# specify complete columns information
#sparse_col = [ 'age',"gender", "DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
sparse_col = ["gender",'Comorbidities','Behavioural','Mental']
dense_col = ["age"]
user_col = ["age", "gender"]
#item_col = ["DEGREE_CENTRALITY","EIGENVECTOR",'BETWEENNESS_CENTRALITY','CLUSTCOEF','PAGERANK','Comorbidities','Behavioural','Mental']
item_col = ['Comorbidities','Behavioural','Mental']



train_data, data_info = DatasetFeat.build_trainset(
    train_data, user_col, item_col, sparse_col, dense_col
)
test_data = DatasetFeat.build_testset(test_data)
train_data.build_negative_samples(data_info)  # sample negative items for each record
test_data.build_negative_samples(data_info)
print(data_info)  # n_users: 5962, n_items: 3226, data sparsity: 0.4185 %
reset_state("DIN")
din = DIN("ranking", data_info, embed_size=16, n_epochs=2,
              recent_num=10, lr=1e-2, lr_decay=False, reg=None,
              batch_size=2048, num_neg=1, use_bn=False, dropout_rate=None,
              hidden_units="128,64,32", tf_sess_config=None, use_tf_attention=True)
din.fit(train_data, verbose=2, shuffle=True, eval_data=test_data,
                metrics=["loss", "roc_auc", "precision", "recall", "map", "ndcg"])

# predict preference of user 2211 to item 110
#print("prediction: ", fm.predict(user=2916, item=1))
# recommend 7 items for user 2916
print("recommendation(id, probability): ", din.recommend_user(user=2916, n_rec=10))

random neg item sampling elapsed: 0.052s
random neg item sampling elapsed: 0.004s
n_users: 19828, n_items: 34, data sparsity: 6.3931 %

 ============================== DIN ==============================
Training start time: 2021-09-22 06:22:12
total params: 351,057 | embedding params: 318,257 | network params: 32,800


train: 100%|██████████| 43/43 [00:01<00:00, 26.55it/s]


Epoch 1 elapsed: 1.659s
	 train_loss: 0.441


eval_rec: 100%|██████████| 2048/2048 [00:02<00:00, 814.83it/s]


	 eval log_loss: 0.4869
	 eval roc_auc: 0.8479
	 eval precision@10: 0.0973
	 eval recall@10: 0.8835
	 eval map@10: 0.6490
	 eval ndcg@10: 0.7118


train: 100%|██████████| 43/43 [00:01<00:00, 30.64it/s]


Epoch 2 elapsed: 1.445s
	 train_loss: 0.2294


eval_rec: 100%|██████████| 2048/2048 [00:02<00:00, 813.68it/s]


	 eval log_loss: 0.4955
	 eval roc_auc: 0.8595
	 eval precision@10: 0.0994
	 eval recall@10: 0.9004
	 eval map@10: 0.6433
	 eval ndcg@10: 0.7110
recommendation(id, probability):  [(34, 0.75017), (26, 0.36038828), (32, 0.3591141), (30, 0.35123292), (31, 0.34588137), (28, 0.31077814), (33, 0.2721323), (23, 0.23841348), (22, 0.15565476), (21, 0.15276368)]
